In [5]:
import torch
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [7]:
batch1 = unpickle('cifar-10-batches-py/data_batch_1')
batch2 = unpickle('cifar-10-batches-py/data_batch_2')
batch3 = unpickle('cifar-10-batches-py/data_batch_3')
batch4 = unpickle('cifar-10-batches-py/data_batch_4')
batch5 = unpickle('cifar-10-batches-py/data_batch_5')
batch_meta = unpickle('cifar-10-batches-py/batches.meta')

In [8]:

df1 = pd.DataFrame(batch1[b'data'])
df2 = pd.DataFrame(batch2[b'data'])
df3 = pd.DataFrame(batch3[b'data'])
df4 = pd.DataFrame(batch4[b'data'])
df5 = pd.DataFrame(batch5[b'data']) 

#load the labels into a dataframe
df1_labels = pd.DataFrame(batch1[b'labels'])
df2_labels = pd.DataFrame(batch2[b'labels'])
df3_labels = pd.DataFrame(batch3[b'labels'])
df4_labels = pd.DataFrame(batch4[b'labels'])
df5_labels = pd.DataFrame(batch5[b'labels'])

In [9]:



#concatenate the dataframes
df = pd.concat([df1, df2, df3, df4, df5], ignore_index=True)
df_labels = pd.concat([df1_labels, df2_labels, df3_labels, df4_labels, df5_labels], ignore_index=True)

#load the data into a tensor

data = torch.from_numpy(df.values)
labels = torch.from_numpy(df_labels.values)

#reshape the data into a 4D tensor
data = data.reshape(50000, 3, 32, 32)

